In [ ]:
# ENIGMA Pre Imputation Code

$HOME/enigma/DTIgenetics/
mkdir -p $HOME/enigma/DTIgenetics/

cd $HOME/enigma/DTIgenetics/

# SNP filtering and Pruning for accurate sex check

plink --bfile ${datafile} --mind 1 --hwe 1e-06 --geno 0.01 --maf 0.05 --make-bed --out ${datafile}_filtered
plink --bfile ${datafile}_filtered --indep-pairphase 20000 2000 0.5 --out ${datafile}_pruned
plink --bfile ${datafile}_filtered --extract ${datafile}_pruned.prune.in --make-bed --out ${datafile}_pruned

# Split-X to deal with pseudo-autosomal regions
awk '{if ($1==25) print $1}' ${datafile}_pruned.bim | wc -l

plink --bfile ${datafile}_pruned --split-x b37 no-fail --make-bed --out ${datafile}_splitX

plink --bfile ${datafile}_splitX --check-sex 0.2 0.8 --out ${datafile}_sexcheck

grep PROBLEM ${datafile}_sexcheck.sexcheck > sex.drop

wc -l sex.drop > ${datafile}_sexcheck_PROBLEM.txt

plink --bfile ${datafile} --remove sex.drop --make-bed --out ${datafile}_QC1

awk '{if ($1=="23") print $0}' ${datafile}_QC1.bim | wc -l > snp_count_X.txt
echo "The number of SNPs on the X chromosome is:"
cat snp_count_X.txt

In [ ]:
# Check for related and duplicated samples
plink --bfile ${datafile}_QC1 --mind 1 --geno 0.01 --maf 0.05 --make-bed --out ${datafile}_QC1tmp
plink --bfile ${datafile}_QC1tmp --indep-pairwise 100 5 0.2 --out ${datafile}_QC1pruned
plink --bfile ${datafile}_QC1tmp --extract ${datafile}_QC1pruned.prune.in --make-bed --out ${datafile}_QC1pruned

plink --bfile ${datafile}_QC1 --remove pihat_duplicates.txt --make-bed --out ${datafile}_QC2

wc -l pihat_duplicates.txt > ${datafile}_QC1pruned_duplicates_count.txt

wc -l pihat_relatedness.genome > ${datafile}_QC1pruned_relatedness_count.txt

In [ ]:
# Multi-Dimensional Scaling (MDS) Protocol
wget https://enigma.ini.usc.edu/website_downloads/ENIGMA_DTI_downloads/HapMap3/HM3_b37.bed.gz
wget https://enigma.ini.usc.edu/website_downloads/ENIGMA_DTI_downloads/HapMap3/HM3_b37.bim.gz
wget https://enigma.ini.usc.edu/website_downloads/ENIGMA_DTI_downloads/HapMap3/HM3_b37.fam.gz

plink --bfile ${datafile}_QC2 --mind 1 --hwe 1e-6 --geno 0.05 --maf 0.01 --make-bed --out ${datafile}_QC2_filtered

gunzip HM3_b37.*
awk '{print $2}' HM3_b37.bim > HM3_b37.snplist.txt
plink --bfile ${datafile}_QC2_filtered --extract HM3_b37.snplist.txt --make-bed --out ${datafile}_QC2local
awk '{ if (($5=="T" && $6=="A")||($5=="A" && $6=="T")||($5=="C" && $6=="G")||($5=="G" && $6=="C")) print $2, "ambig"; else print $2 ;}' ${datafile}_QC2_filtered.bim | grep -v ambig > local.snplist.txt
plink --bfile HM3_b37 --extract local.snplist.txt --make-bed --out HM3_b37_external

plink --bfile ${datafile}_QC2local --bmerge HM3_b37_external.bed HM3_b37_external.bim HM3_b37_external.fam --make-bed --out ${datafile}_QC2local_HM3b37merge

plink --bfile ${datafile}_QC2local --flip ${datafile}_QC2local_HM3b37merge-merge.missnp --make-bed --out ${datafile}_QC2local_flipped
plink --bfile ${datafile}_QC2local_flipped --bmerge HM3_b37_external.bed HM3_b37_external.bim HM3_b37_external.fam --make-bed --out ${datafile}_QC2local_HM3b37merge

plink --bfile ${datafile}_QC2local_flipped --exclude  ${datafile}_QC2local_HM3b37merge-merge.missnp --make-bed --out ${datafile}_QC2local_flipped2
plink --bfile ${datafile}_QC2local_flipped2 --bmerge HM3_b37_external.bed HM3_b37_external.bim HM3_b37_external.fam --make-bed --out ${datafile}_QC2local_HM3b37merge

In [ ]:
# Run the MDS analysis
plink --bfile ${datafile}_QC2local_HM3b37merge --cluster --mind .05 --mds-plot 10 --extract local.snplist.txt --out ${datafile}_QC2_HM3b37mds

# Plot the MDS results using R
cat ${datafile}_QC2_HM3b37mds.mds | tr -s ' ' '\t' > ${datafile}_QC2_HM3b37.mds.tsv
awk 'BEGIN{OFS=","};{print $1, $2, $3, $4, $5, $6, $7}' >> ${datafile}_QC2_HM3b37mds2R.mds.csv ${datafile}_QC2_HM3b37mds.mds

module load R 

R 

#In R
library(calibrate)
#If you don’t have the calibrate package, install it using
#install.packages("calibrate")
mds.cluster = read.csv("DATAFILE_QC2_HM3b37mds2R.mds.csv", header=T);
#first we create an additional column holding the respective ancestry labels
mds.cluster$POP=rep("DATASETNAME",length(mds.cluster$C1));
mds.cluster$POP[which(mds.cluster$FID == "CEU")] <- "CEU";
mds.cluster$POP[which(mds.cluster$FID == "CHB")] <- "CHB";
mds.cluster$POP[which(mds.cluster$FID == "YRI")] <- "YRI";
mds.cluster$POP[which(mds.cluster$FID == "TSI")] <- "TSI";
mds.cluster$POP[which(mds.cluster$FID == "JPT")] <- "JPT";
mds.cluster$POP[which(mds.cluster$FID == "CHD")] <- "CHD";
mds.cluster$POP[which(mds.cluster$FID == "MEX")] <- "MEX";
mds.cluster$POP[which(mds.cluster$FID == "GIH")] <- "GIH";
mds.cluster$POP[which(mds.cluster$FID == "ASW")] <- "ASW";
mds.cluster$POP[which(mds.cluster$FID == "LWK")] <- "LWK";
mds.cluster$POP[which(mds.cluster$FID == "MKK")] <- "MKK";

#get the colors for plotting
colors=rep("red",length(mds.cluster$C1));
colors[which(mds.cluster$POP == "CEU")] <- "lightblue";
colors[which(mds.cluster$POP == "CHB")] <- "brown";
colors[which(mds.cluster$POP == "YRI")] <- "yellow";
colors[which(mds.cluster$POP == "TSI")] <- "green";
colors[which(mds.cluster$POP == "JPT")] <- "purple";
colors[which(mds.cluster$POP == "CHD")] <- "orange";
colors[which(mds.cluster$POP == "MEX")] <- "grey50";
colors[which(mds.cluster$POP == "GIH")] <- "black";
colors[which(mds.cluster$POP == "ASW")] <- "darkolivegreen";
colors[which(mds.cluster$POP == "LWK")] <- "magenta";
colors[which(mds.cluster$POP == "MKK")] <- "darkblue";

pdf(file="mdsplot_DATASETNAME_QC2_outliersincluded.pdf",width=7,height=7)

#Please note - the plot will directly be saved as a PDF in your current working directory under the above specified name
#and does not appear in the PLOT window
plot(rev(mds.cluster$C2), rev(mds.cluster$C1), col=rev(colors), ylab="Dimension 1", xlab="Dimension 2",pch=20)
minor.tick(nx = 5, ny = 5,   # Ticks density
         tick.ratio = 0.5) # Ticks size
legend("bottomleft", c("DATASETNAME", "CEU", "CHB", "YRI", "TSI", "JPT", "CHD", "MEX", "GIH", "ASW","LWK", "MKK"), fill=c("red", "lightblue", "brown", "yellow", "green", "purple", "orange", "grey50", "black", "darkolivegreen", "magenta", "darkblue"))
dev.off();
#If you want to know the subject ID label of each sample on the graph, you can label your sample points by uncommenting the commands below.
#This is optional and you can choose not to do this if you are worried about patient information being sent; when you send us your MDS plot please make sure the subject ID labels are NOT on the graph.
#POPlabels <- c("CEU", "CHB", "YRI", "TSI", "JPT", "CHD", "MEX", "GIH", "ASW", "LWK", "MKK");
#textxy(mds.cluster[which(!(mds.cluster$POP %in% POPlabels)), "C2"], mds.cluster[which(!(mds.cluster$POP %in% POPlabels)), "C1"], mds.cluster[which(!(mds.cluster$POP %in% POPlabels)), "IID"])

# Identify Ancestry Outliers

MDS_mySample <- mds.cluster[which(mds.cluster$POP %in% c("`DATASETNAME")),]
MDS_mySample$outlier  <- 0
# calculate outliers - first a threshold example for the g1000mix, a quite homogenous sample
# NOTE: change the cut-off below depending on YOUR sample 
MDS_mySample$outlier[MDS_mySample$C1 < -0.01 | MDS_mySample$C2 > -0.05] <- 1

MDS_mySample_outliers<- MDS_mySample[which(MDS_mySample$outlier == 1),1:2]
MDS_mySample_european<- MDS_mySample[which(MDS_mySample$outlier == 0 ),1:2]
#For further plotting we also grab the full reference details again and combine them with the full EU information in your sample
#First let's get the reference - we get everything not part of your dataset
MDS_ref_eur <- mds.cluster[which "(!(mds.cluster$POP %in% "DATASETNAME")),]
#combine this with all Europeans (or respective other ancestry cohort)
MDS_mySample_eur_plus_ref <- rbind(MDS_ref_eur, mds.cluster[which (mds.cluster$IID %in% MDS_mySample_european$IID ),])
# repeat the colours again for the updated dataframe and then plot - note again: plot is directly saved
colors=rep("red",length(MDS_mySample_eur_plus_ref$C1));
colors[which(MDS_mySample_eur_plus_ref$POP == "CEU")] <- "lightblue";
colors[which(MDS_mySample_eur_plus_ref$POP == "CHB")] <- "brown";
colors[which(MDS_mySample_eur_plus_ref$POP == "YRI")] <- "yellow";
colors[which(MDS_mySample_eur_plus_ref$POP == "TSI")] <- "green";
colors[which(MDS_mySample_eur_plus_ref$POP == "JPT")] <- "purple";
colors[which(MDS_mySample_eur_plus_ref$POP == "CHD")] <- "orange";
colors[which(MDS_mySample_eur_plus_ref$POP == "MEX")] <- "grey50";
colors[which(MDS_mySample_eur_plus_ref$POP == "GIH")] <- "black";
colors[which(MDS_mySample_eur_plus_ref$POP == "ASW")] <- "darkolivegreen";
colors[which(MDS_mySample_eur_plus_ref$POP == "LWK")] <- "magenta";
colors[which(MDS_mySample_eur_plus_ref$POP == "MKK")] <- "darkblue";

pdf(file="mdsplot_DATASETNAME_QC2_outliersExcluded.pdf", width=7, height=7)
plot(rev(MDS_mySample_eur_plus_ref$C2), rev(MDS_mySample_eur_plus_ref$C1), col=rev(colors), ylab="Dimension 1", xlab="Dimension 2", pch=20)
legend("bottomleft", c("DATASETNAME", "CEU", "CHB", "YRI", "TSI", "JPT", "CHD", "MEX", "GIH", "ASW","LWK", "MKK"), fill=c("red", "lightblue", "brown", "yellow", "green", "purple", "orange", "grey50", "black", "darkolivegreen", "magenta",` `"darkblue"))
# save files - don't forget to replace DATASETNAME with your cohort-specific abbreviation
#let's get the outliers in your sample
write.table(MDS_mySample_outliers, "DATASETNAME_pop_strat_mds.outlier.txt", sep="\t", quote=FALSE, row.names=FALSE)
#let's get the europeans in your sample
write.table(MDS_mySample_european, "DATASETNAME_pop_strat_mds.eur.txt", sep="\t", quote=FALSE, row.names=FALSE)
# quit R
q()

plink --bfile ${datafile}_QC2_filtered  --keep ${datafile}_pop_strat_mds.eur.txt --make-bed --out ${datafile}_QC3

In [ ]:
# Get genetic principal components in ancestry homogenous group
plink --bfile ${datafile}_QC3 --pca --extract local.snplist.txt --out ${datafile}_PCACovariates

R

library(data.table)
library(ggplot2)

#read in the eigenvalue file
eigv <- fread("${datafile}_PCACovariates.eigenval")
#calculate variance explained
var_explained <- (eigv/(sum(eigv))*100
#plot the scree plot
qplot(c(1:20), var_explained$V1) +
geom_line() +
xlab("Principal Component") +
ylab("Variance Explained") +
ggtitle("Scree Plot") +
ylim(0, 100)

In [ ]:
# Cohort Summary Data 

# Pre-QC subject numbers
echo "COHORTNAME N_CONTROLS N_CASES N_CONTROLS_M N_CONTROLS_F N_CASES_M N_CASES_F PROP_CONTROLS_F PROP_CONTROLS_M PROP_CASES_F PROP_CASES_M" > ${datafile}_basic_stats_preQC.txt
echo -n "${datafile} " >> ${datafile}_basic_stats_preQC.txt
#How many controls/unaffected are in the cohort?
N_CONTROLS=`awk '{if ($6==1) print $0}' ${datafile}.fam | wc -l`
echo -n "$N_CONTROLS " >> ${datafile}_basic_stats_preQC.txt
#How many patients are in the cohort?
N_CASES=`awk '{if ($6==2) print $0}' ${datafile}.fam | wc -l`
echo -n "$N_CASES " >> ${datafile}_basic_stats_preQC.txt
#What is the absolute number of males among controls/unaffected?
N_CONTROLS_M=`awk '{if ($6==1 && $5==1) print $0}' ${datafile}.fam | wc -l`
echo -n "$N_CONTROLS_M " >> ${datafile}_basic_stats_preQC.txt
#What is the absolute number of females among controls/unaffected?
N_CONTROLS_F=`awk '{if ($6==1 && $5==2) print $0}' ${datafile}.fam | wc -l`
echo -n "$N_CONTROLS_F " >> ${datafile}_basic_stats_preQC.txt
#What is the absolute number of males among patients?
N_CASES_M=`awk '{if ($6==2 && $5==1) print $0}' ${datafile}.fam | wc -l`
echo -n "$N_CASES_M " >> ${datafile}_basic_stats_preQC.txt
#What is the absolute number of females among patients?
N_CASES_F=`awk '{if ($6==2 && $5==2) print $0}' ${datafile}.fam | wc -l`
echo -n "$N_CASES_F " >> ${datafile}_basic_stats_preQC.txt
#What is the proportion of females among controls/unaffected?
PROP_CONTROLS_F=`echo "$N_CONTROLS_F/$N_CONTROLS" | bc -l`
#What is the proportion of males among controls/unaffected?
PROP_CONTROLS_M=`echo "$N_CONTROLS_M/$N_CONTROLS" | bc -l`
#What is the proportion of females among patients?
PROP_CASES_F=`if (($N_CASES == 0)); then echo 0; else echo "$N_CASES_F/($N_CASES_F+$N_CASES_M)" | bc -l; fi`
#What is the proportion of males among patients?
PROP_CASES_M=`if (($N_CASES == 0)); then echo 0; else echo "$N_CASES_M/($N_CASES_F+$N_CASES_M)" | bc -l; fi`
echo -n "$PROP_CONTROLS_F $PROP_CONTROLS_M $PROP_CASES_F $PROP_CASES_M " >> ${datafile}_basic_stats_preQC.txt


# SNP level QC

echo "COHORTNAME N_SNPs_preQC N_samples_preQC SNPs_removed_>missingnessT Samples_removed_>missingnessT SNPs_removed_<MAFt SNPs_removed_HWE Samples_removed_MDS_outliers N_SNPs_postQC N_samples_postQC" > ${datafile}_qc_summary.txt
echo -n "${datafile} " >> ${datafile}_qc_summary.txt
NLINES=`wc -l ${datafile}.bim | awk '{print $1}'`
echo -n $NLINES" " >> ${datafile}_qc_summary.txt
NLINES=`wc -l ${datafile}.fam | awk '{print $1}'`
echo -n $NLINES" " >> ${datafile}_qc_summary.txt
qc_VAR=$(grep "variants removed due to missing genotype data (--geno)" ${datafile}_QC2_filtered.log | awk '{printf $1}')
if [ -z "$qc_VAR" ];then echo -n "NaN"" " >> ${datafile}_qc_summary.txt; else echo -n "$qc_VAR"" " >> ${datafile}_qc_summary.txt;fi
qc_VAR=$(grep "people removed due to missing genotype data (--mind)" ${datafile}_QC2_filtered.log | awk '{printf $1}')
if [ -z "$qc_VAR" ];then echo -n "NaN"" " >> ${datafile}_qc_summary.txt; else echo -n "$qc_VAR"" " >> ${datafile}_qc_summary.txt;fi
qc_VAR=$(grep "variants removed due to minor allele threshold(s)" ${datafile}_QC2_filtered.log | awk '{printf $1}')
if [ -z "$qc_VAR" ];then echo -n "NaN"" " >> ${datafile}_qc_summary.txt; else echo -n "$qc_VAR"" " >> ${datafile}_qc_summary.txt;fi
qc_VAR=$(grep "variants removed due to Hardy-Weinberg exact test" ${datafile}_QC2_filtered.log | awk '{printf $2}')
if [ -z "$qc_VAR" ];then echo -n "NaN"" " >> ${datafile}_qc_summary.txt; else echo -n "$qc_VAR"" " >> ${datafile}_qc_summary.txt;fi
NLINES2=`wc -l ${datafile}_QC2.bim | awk '{print $1}'`
NLINES3=`wc -l ${datafile}_QC3.bim | awk '{print $1}'`
N_excluded_MDS_outliers=`echo "$NLINES2-$NLINES3" | bc -l`
echo -n $N_excluded_MDS_outliers" " >> ${datafile}_qc_summary.txt
NLINES=`wc -l ${datafile}_QC3.bim | awk '{print $1}'`
echo -n $NLINES" " >> ${datafile}_qc_summary.txt
NLINES=`wc -l ${datafile}_QC3.fam | awk '{print $1}'`
echo -n $NLINES" " >> ${datafile}_qc_summary.txt

# Post-QC subject numbers

echo "COHORTNAME N_CONTROLS N_CASES N_CONTROLS_M N_CONTROLS_F N_CASES_M N_CASES_F PROP_CONTROLS_F PROP_CONTROLS_M PROP_CASES_F PROP_CASES_M" > ${datafile}_basic_stats_postQC.txt
echo -n "${datafile} " >> ${datafile}_basic_stats_postQC.txt
#How many controls/unaffected are in the cohort?
N_CONTROLS=`awk '{if ($6==1) print $0}' ${datafile}_QC3.fam | wc -l`
echo -n "$N_CONTROLS " >> ${datafile}_basic_stats_postQC.txt
#How many patients are in the cohort?
N_CASES=`awk '{if ($6==2) print $0}' ${datafile}_QC3.fam | wc -l`
echo -n "$N_CASES " >> ${datafile}_basic_stats_postQC.txt
#What is the absolute number of males among controls/unaffected?
N_CONTROLS_M=`awk '{if ($6==1 && $5==1) print $0}' ${datafile}_QC3.fam | wc -l`
echo -n "$N_CONTROLS_M " >> ${datafile}_basic_stats_postQC.txt
#What is the absolute number of females among controls/unaffected?
N_CONTROLS_F=`awk '{if ($6==1 && $5==2) print $0}' ${datafile}_QC3.fam | wc -l`
echo -n "$N_CONTROLS_F " >> ${datafile}_basic_stats_postQC.txt
#What is the absolute number of males among patients?
N_CASES_M=`awk '{if ($6==2 && $5==1) print $0}' ${datafile}_QC3.fam | wc -l`
echo -n "$N_CASES_M " >> ${datafile}_basic_stats_postQC.txt
#What is the absolute number of females among patients?
N_CASES_F=`awk '{if ($6==2 && $5==2) print $0}' ${datafile}_QC3.fam | wc -l`
echo -n "$N_CASES_F " >> ${datafile}_basic_stats_postQC.txt
#What is the proportion of females among controls/unaffected?
PROP_CONTROLS_F=`echo "$N_CONTROLS_F/$N_CONTROLS" | bc -l`
#What is the proportion of males among controls/unaffected?
PROP_CONTROLS_M=`echo "$N_CONTROLS_M/$N_CONTROLS" | bc -l`
#What is the proportion of females among patients?
PROP_CASES_F=`if (($N_CASES == 0)); then echo 0; else echo "$N_CASES_F/($N_CASES_F+$N_CASES_M)" | bc -l; fi`
#What is the proportion of males among patients?
PROP_CASES_M=`if (($N_CASES == 0)); then echo 0; else echo "$N_CASES_M/($N_CASES_F+$N_CASES_M)" |  bc -l; fi`
echo -n "$PROP_CONTROLS_F $PROP_CONTROLS_M $PROP_CASES_F $PROP_CASES_M " >> ${datafile}_basic_stats_postQC.txt